In [157]:
import numpy as np
import math
import residuals as r
np.set_printoptions(threshold=40)

In [193]:
# I am defining the weighted standard deviation according to this definition:
# https://en.wikipedia.org/wiki/Weighted_arithmetic_mean#Weighted_sample_variance
# I am using the inverse square of the error arrays to define the weights

factor10 = math.sqrt((25*60)/10) 
factor80 = math.sqrt((25*60)/80) 



def weighted_stddev(values, errors):
    '''
    Return the weighted average and standard deviation.

    '''
    weights = 1 / ((errors)**2)
    average = np.average(values, weights=weights)
    variance = np.average((values-average)**2, weights=weights)  # Fast and numerically precise
    return (math.sqrt(variance))



def jitter_ratio(dumpresids_file, condition_lower, condition_upper, titlearg):
    '''
    Return the ratio of sigma_dump over sigma_all.

    '''
    
    #Define sigma_all (or std_all) as entire observation period
    x=r.read_residuals(filename=dumpresids_file)

    condition = (x.bary_TOA > condition_lower) & (x.bary_TOA < condition_upper)
    Resid_all = x.prefit_sec[condition]
    Errors_all = x.uncertainty[condition]
    std_all = weighted_stddev(Resid_all, Errors_all)
    
    ##============
    
    #Define sigma_dump (or std_all) as individual dumps within observation period
    
    #This isolates the numbers for the different dump times
    rounded = np.round(np.array([x.bary_TOA[condition] - 0.00005]), 4)
    dumptimes=[]
    for i in rounded.tolist()[0]: 
        if i not in dumptimes:
            dumptimes.append(i)

    #This calculates a list of sigma_dump values and averages those
    dump_devs=[]
    for i in dumptimes:
        timeindex = (rounded == i)[0]
        Resid = Resid_all[timeindex]
        Errors = Errors_all[timeindex]
        single_std_dump = weighted_stddev(Resid, Errors)
        dump_devs.append(single_std_dump)
    std_dump=np.average(dump_devs)
    
    #Setup jitter ratio
    ratio = std_all / std_dump
    
    print titlearg
    print std_dump
    print "jitter ratio = " + str(ratio)   

In [92]:
#Checking order of operations

print 25*60
print 1500/10
print math.sqrt(150)
print math.sqrt((25*60)/10)

print math.sqrt((25*60)/80)

#yes, order of operations works

1500
150
12.2474487139
12.2474487139
4.24264068712


In [194]:
jitter_ratio("/nimrod1/eschwab/residuals/J1713+0747_resid_56380_80F8.tmp",
            56380.35, 56380.367, 'J1713+0747, MJD 56380, 80 second subintervals')

J1713+0747, MJD 56380, 80 second subintervals
2.49624000145e-08
jitter ratio = 9.79134961678


In [195]:
jitter_ratio("/nimrod1/eschwab/residuals/J1713+0747_resid_56380_NTF8.tmp",
            56380.35, 56380.367, 'J1713+0747, MJD 56380, 10 second subintervals')

J1713+0747, MJD 56380, 10 second subintervals
7.10113781891e-08
jitter ratio = 8.20533075938


In [196]:
jitter_ratio("/nimrod1/eschwab/residuals/J1713+0747_resid_56201_80F8.tmp",
            56201.85, 56201.872, 'J1713+0747, MJD 56201, 80 second subintervals')

J1713+0747, MJD 56201, 80 second subintervals
5.08948679605e-08
jitter ratio = 3.40901238215


In [197]:
jitter_ratio("/nimrod1/eschwab/residuals/J1713+0747_resid_56201_NTF8.tmp",
            56201.85, 56201.872, 'J1713+0747, MJD 56201, 10 second subintervals')

J1713+0747, MJD 56201, 10 second subintervals
8.13611454109e-08
jitter ratio = 6.67670588045


In [198]:
jitter_ratio("/nimrod1/eschwab/residuals/B1937+21_resid_56465_80F8.tmp",
            56465.3, 56465.32, 'B1937+21, MJD 56465, 80 second subintervals')

B1937+21, MJD 56465, 80 second subintervals
4.44210286622e-08
jitter ratio = 1.05368813293


In [199]:
jitter_ratio("/nimrod1/eschwab/residuals/B1937+21_resid_56465_NTF8.tmp",
            56465.3, 56465.32, 'B1937+21, MJD 56465, 10 second subintervals')

B1937+21, MJD 56465, 10 second subintervals
4.92378535069e-08
jitter ratio = 1.43210742163
